<img src='logo.png' style='display: block;height: 61px;float: left;padding: .75rem 1.25rem;'>

# Desafío - Naive Bayes
* Para realizar este desafío debes haber revisado la lectura y videos correspondiente a la unidad.
* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook y csv).
* Una vez terminado el desafío, comprime la carpeta y sube el `.zip` a la seccióncorrespondiente.

## Descripción

* En esta sesión trabajaremos con una serie de base de datos sobre letras musicales de distintos artistas. Cada uno de los `csv` se encuentra en la carpeta `dump` dentro del proyecto.
* Cada csv tiene el nombre del artista a analizar. Los archivos contienen el nombre del artista, el género musical del artista, el nombre de la canción y las letras.
* En base a esta información, se les pide un modelo generativo que pueda predecir el género de una canción a patir de la letra de una canción.
* Existen 4 géneros que se registran en la base de datos, por lo que para esta actividad trabajaremos con un Multinomial Naive Bayes.

Ejercicio 1: Preparar el ambiente de trabajo

___
* Importe los módulos `numpy` , `pandas` , `matplotlib` , `seaborn` , `glob` y `os` siguiendo las buenas prácticas. Los últimos dos módulos permitirán realizar la importación de múltiples archivos dentro de la carpeta `dump`.

<div style='color: #004085; background-color: #cce5ff;border-color: #b8daff;border-radius: .25rem;padding: .75rem 1.25rem;'>
Se utilizan librerías bases para el tratamiento de datos y algunos aspectos básicos de cálculo y gráficos.
</div>

In [3]:
# Se importa libreria para el manejo de bases de datos
import pandas as pd
# Se importa libreria para el manejo de operaciones de cálculo
import numpy as np
# Se importa libreria para el manejo de gráficos
import matplotlib.pyplot as plt
# Se importa libreria para manejo de funciones estadisticas y probabilidades
import seaborn as sns
# Librería para visualizar patrones de datos perdidos
import missingno as msngo
# Se importa libreria para el menejo de warning
import warnings
# Se importa libreria para el menejo de expresiones regulares
import glob, os

<div style='color: #004085; background-color: #cce5ff;border-color: #b8daff;border-radius: .25rem;padding: .75rem 1.25rem;'>
Definimos algunos aspectos de ambiente y valores por defecto.
</div>

In [4]:
# Por defecto, matplotlib crea una figura en una ventana separada.
# podemos hacer que las figuras aparezcan en línea dentro del notebook; lo hacemos ejecutando:
%matplotlib inline
# Se ignoran los warning para evitar suciedad en la ejecución
warnings.filterwarnings(action='ignore')
# Se define el estilo de gráficos a usar
plt.style.use('seaborn')
# Se define el tamaño de los paños de los gráficos por defecto
plt.rcParams['figure.figsize'] = (10, 6)
# Dado que vamos a supervisar datos no limitaremos la cantidad de columnas a mostrar en el despliegue del dataframe
pd.options.display.max_columns = None

<div style='color: #004085; background-color: #cce5ff;border-color: #b8daff;border-radius: .25rem;padding: .75rem 1.25rem;'>
Se utiliza <strong>librería propia</strong> que continen funciones auxiliares, necesarias para el desarrollo de desafíos.
</div>

In [5]:
# Se importa libreria con funciones auxiliares
import ancilliary_funcs as afx

<div style='color: #004085; background-color: #cce5ff;border-color: #b8daff;border-radius: .25rem;padding: .75rem 1.25rem;'>
Se utilizan librerías <strong>sklearn</strong> para el tratamiento de escala de valores de los atributos y seleccion de set de datos:
</div>

In [34]:
# Método para la selección de datos entre entrenamiento y test
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

* Para ello genere un objeto que guarde en una lista todos los archivos alojados en `dump` utilizando `glob.glob` y `os.getcwd` para extraer las rutas absolutas. Posteriormente genere un objeto `pd.DataFrame` que contenga todos los csv.
* Asegúrese de eliminar la columna `Unnamed: 0` que se genera por defecto.

In [18]:
file_list = glob.glob(os.getcwd() + '/dump/*.csv')

In [24]:
append_csv = []

for filename in file_list:
    append_csv.append(pd.read_csv(filename, index_col=None, header=0)
                      .drop(columns = 'Unnamed: 0'))

In [31]:
df = pd.concat(append_csv)
df.columns = ['Artis', 'Genre', 'Song', 'Lyrics']

In [32]:
df.sample(5)

,Artis,Genre,Song,Lyrics
83,Bruce Springsteen,rock,Jersey Girl,I got no time for the corner boys \n Down in t...
15,Pink Floyd,rock,It Would Be So Nice,It would be so nice \n It would be so nice \n ...
42,Mayhem,metal,Illuminate Eliminate,"Where I came from... I must return \n No more,..."
89,Weezer,rock,Everyone,"Unfortunately, we are not licensed to display ..."
33,Bruce Springsteen,rock,Darkness On The Edge Of Town,They're still racing out at the Trestles \n Bu...


In [33]:
# Revisamos las dimensiones de la base de datos
df.shape

(9489, 4)

<div style='color: #004085; background-color: #cce5ff;border-color: #b8daff;border-radius: .25rem;padding: .75rem 1.25rem;'>
La base de datos se compone de 9489 observaciones con 4 atributos. 
</div>

## Ejercicio 2: Descripción de los datos
___
* Utilizando el objeto creado en el Ejercicio 1, genere dos gráficos de barras que resuman la siguiente información:
    - La cantidad de canciones registradas por cada artista, ordenado de mayor a menor.
    - La cantidad de canciones registradas en cada género, ordenados de mayor a menor.
* Comente sobre las principales tendencias.

## Ejercicio 3: Matriz de ocurrencias


__Digresión: Tokenización de Textos__

Para poder trabajar con textos, debemos pasarlos a una __matriz dispersa__, donde cada fila representará una entrada (en este caso, una canción), y cada columna __representará una palabra (token)__. Este es el proceso de tokenización: Identificar la ocurrencia de una palabra específica dentro de un conjunto de textos (corpus). El tokenizador más simple
`sklearn.feature_extraction.text.CountVectorizer` genera una colección de textos a una matriz que representa la frecuencia __dentro del texto__ de una palabra específica. El tokenizador funciona de la siguiente manera:


* Importe la clase `CountVectorizer` dentro de los módulos `feature_extraction.text` de la librería `sklearn`. Lea la documentación asociada a ésta. ¿Cuál es el objetivo de esta clase?

In [35]:
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer_fit = count_vectorizer.fit_transform(df['Lyrics'])

* Aplique la clase para extraer las 100 palabras más repetidas en toda la base de datos.

In [82]:
words = count_vectorizer.get_feature_names()
words_freq = count_vectorizer_fit.toarray().sum(axis=0)
df_words = pd.DataFrame({'word': words, 'freq': words_freq})
df_words_100 = df_words.sort_values(by='freq', ascending=False).head(100)
df_words_100

,word,freq
24950,like,19629
12859,don,17398
23856,know,14962
18439,got,14171
23287,just,13978
25479,love,11268
48591,yeah,11071
25165,ll,10028
29851,oh,9879
7009,cause,8356


* Genere una función que replique el procedimiento para cada uno de los géneros.

In [78]:
def Ocurrencia(df, column, value, top = 100):
    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer_fit = count_vectorizer.fit_transform(df[df[column]==value]['Lyrics'])
    words = count_vectorizer.get_feature_names()
    words_freq = count_vectorizer_fit.toarray().sum(axis=0)
    df_words = pd.DataFrame({'word': words, 'freq': words_freq})
    df_words_100 = df_words.sort_values(by='freq', ascending=False).head(100)
    return df_words_100

In [81]:
for gen in df['Genre'].unique():
    print(Ocurrencia(df, 'Genre', gen, top = 100))

         word   freq
18361    like  12055
13533     got   7613
9466      don   7604
17532    know   6535
17082    just   6272
28298    shit   5546
909       ain   4782
5237    cause   4737
19198     man   4481
35793      yo   4425
35721    yeah   4087
21434   nigga   3993
12661    fuck   3705
18202     let   3436
18531      ll   3391
19137    make   3099
10400      em   3078
6578     come   2992
32299    time   2982
26442   right   2936
27334     say   2836
18783    love   2830
35636      ya   2765
21439  niggas   2666
21975      oh   2375
34582    want   2360
34576   wanna   2230
18308    life   2143
3197    bitch   2123
1821      ass   2054
...       ...    ...
33080     try   1013
14824    high    987
26955     run    986
14787     hey    985
35421    word    978
14313    hard    971
11238    face    970
14531    hear    967
29513     son    959
2255      bad    939
23749    play    935
21454   night    924
18074   leave    919
25459  really    916
18649    long    915
8039     damn

<div style='color: #854000; background-color: #ffe5e5;border-color: #ffdaff;border-radius: .25rem;padding: .75rem 1.25rem;'>
</div>